In [1]:
import networkx as nx
import osmnx as ox
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from multiprocessing import Pool

# Getting Distances

In this notebook we will work the script to get the network distances from prepared (and trimmed) OD matrices

## 1. OSMnx modified distance functions:

The following functions are used under the hood by OSMNx to get shortest paths. I have to modify them in order to retrieve not the shortest path but the shortest path length. This could be done directly, without appealing to OSMnx, with NetworkX. However, this modification is easy and is already parallelized---so that we can run the computations with multiple cores.

In [2]:
def _single_shortest_path_distance(G, orig, dest, weight):
    """
    Get shortest path distance from an origin node to a destination node.
    This function is a convenience wrapper around networkx.shortest_path, with
    exception handling for unsolvable paths.
    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    orig : int
        origin node ID
    dest : int
        destination node ID
    weight : string
        edge attribute to minimize when solving shortest path
    Returns
    -------
    dist : float
        shortest (weighted) distance between origin and destination nodes
    """
    try:
        return nx.shortest_path_length(G, orig, dest, weight=weight) #change function here from G. Boeing's repo
    except nx.exception.NetworkXNoPath:
        return None

def shortest_path_distance(G, orig, dest, weight="length", cpus=1):
    """
    Get shortest path distance from origin node(s) to destination node(s).
    If `orig` and `dest` are single node IDs, this will return a list of the
    nodes constituting the shortest path between them.  If `orig` and `dest`
    are lists of node IDs, this will return a list of lists of the nodes
    constituting the shortest path between each origin-destination pair. If a
    path cannot be solved, this will return None for that path. You can
    parallelize solving multiple paths with the `cpus` parameter, but be
    careful to not exceed your available RAM.
    Parameters
    ----------
    G : networkx.MultiDiGraph
        input graph
    orig : int or list
        origin node ID, or a list of origin node IDs
    dest : int or list
        destination node ID, or a list of destination node IDs
    weight : string
        edge attribute to minimize when solving shortest path
    cpus : int
        how many CPU cores to use; if None, use all available
    Returns
    -------
    path : list
        list of node IDs constituting the shortest path, or, if orig and dest
        are lists, then a list of path lists
    """
    if not (hasattr(orig, "__iter__") or hasattr(dest, "__iter__")):
        # if neither orig nor dest is iterable, just return the shortest path
        return _single_shortest_path_distance(G, orig, dest, weight)

    elif hasattr(orig, "__iter__") and hasattr(dest, "__iter__"):
        # if both orig and dest are iterables ensure they have same lengths
        if len(orig) != len(dest):  # pragma: no cover
            raise ValueError("orig and dest must contain same number of elements")

        if cpus == 1:
            # if single-threading, calculate each shortest path one at a time
            paths = [_single_shortest_path_distance(G, o, d, weight) for o, d in zip(orig, dest)]
        else:
            # if multi-threading, calculate shortest paths in parallel
            args = ((G, o, d, weight) for o, d in zip(orig, dest))
            pool = Pool(cpus)
            sma = pool.starmap_async(_single_shortest_path_distance, args)
            paths = sma.get()
            pool.close()
            pool.join()

        return paths

    else:
        # if only one of orig or dest is iterable and the other is not
        raise ValueError("orig and dest must either both be iterable or neither must be iterable")

## 2. Getting the naive distance:

People walk to amenities less than 2km away and drive to those further. To start picking the mode, we can do a first straight line distance filter. The following function does this job:

In [3]:
def get_naive_OD(fua_code, threshold=2000, number_of_cores=1):
    #Get the files:
    od_matrix = pd.read_csv('../data/d02_processed-safegraph/trimmed-OD-per-FUA/'+fua_code+'_trimmed-ODmatrix.csv')
    walk_graph = ox.project_graph(ox.load_graphml('../data/d03_intermediate/FUA-networks/walk/'+fua_code+'.graphml'), to_crs='EPSG:5070')
    drive_graph = ox.project_graph(ox.load_graphml('../data/d03_intermediate/FUA-networks/drive/'+fua_code+'.graphml'), to_crs='EPSG:5070')
    print('  got all files')

    #Get the geometries of origin and destinations:
    places_pt = gpd.points_from_xy(x= od_matrix.longitude, y=od_matrix.latitude, crs='EPSG:4326').to_crs('EPSG:5070')
    centroids_pt = gpd.points_from_xy(x= od_matrix.intptlon, y=od_matrix.intptlat, crs='EPSG:4326').to_crs('EPSG:5070')

    od_matrix['origin_x'], od_matrix['origin_y'] = centroids_pt.x, centroids_pt.y
    od_matrix['dest_x'], od_matrix['dest_y'] = places_pt.x, places_pt.y

    #Get the Boolean value of whether we walk or drive:
    od_matrix['walk'] = places_pt.distance(centroids_pt) <= threshold
    print('  got preferred mode')

    #Now we split the dataframe into two (one for walking and one for driving):
    od_matrix_dict = {walk: df for walk, df in od_matrix.groupby('walk')}
    G = {False: drive_graph, True: walk_graph}

    #For each of those dataframes, we do nearest nodes from OSMnx on the appropriate graph and the distance:
    full_dfs = []
    for walk, df in od_matrix_dict.items():
        df['origin_node'] = ox.nearest_nodes(G[walk], df['origin_x'], df['origin_y'])
        df['destination_node'] = ox.nearest_nodes(G[walk], df['dest_x'], df['dest_y'])
        df['distance'] = shortest_path_distance(G[walk],
                                                df['origin_node'].values, df['destination_node'].values,
                                                cpus=number_of_cores)
        full_dfs.append(df)

    merged_df = pd.concat(full_dfs, ignore_index=True)
    expanded_OD = merged_df.drop(['origin_x', 'origin_y', 'dest_x', 'dest_y'], axis=1)
    print('  got expanded matrix')

    expanded_OD.to_csv('../data/d04_final-OD-matrices/naive-OD-per-FUA/' + fua_code+'_full-ODmatrix.csv')
    print('  saved')

This code can be ran in the cluster through the executable script get-distance.py . We can submit one job per FUA with its code.

## 3. Evaluating the Naive Distance:

After running the script (which takes less than an hour for most FUAs but up to 15 hours for the largest ones) we obtained all the naive OD matrices with distance. Now it is time to go back to verify the access modes (walk vs. drive) are coherent with the network distance computed. Let's load a few files:

In [4]:
def get_files(fua_code):
    
    buffered_boundary = gpd.read_file('../data/d03_intermediate/FUA-buffered-shapefile/FUA-buffered.shp').set_index('fuacode').loc[[fua_code]]
    walk_graph = ox.load_graphml('../data/d03_intermediate/FUA-networks/walk/'+fua_code+'.graphml')
    drive_graph = ox.load_graphml('../data/d03_intermediate/FUA-networks/drive/'+fua_code+'.graphml')
    OD_matrix = pd.read_csv('../data/d04_final-OD-matrices/naive-OD-per-FUA/'+fua_code+'_full-ODmatrix.csv').drop('Unnamed: 0', axis=1)
    
    return buffered_boundary, walk_graph, drive_graph, OD_matrix

In [5]:
buffered_boundary_80, walk_graph_80, drive_graph_80, OD_matrix_80 = get_files('USA80')
buffered_boundary_81, walk_graph_81, drive_graph_81, OD_matrix_81 = get_files('USA81')

In some cases, even though the commute was made "through walking", the distance is superior to 2,000 meters. This is because the walk vs. drive original split was made in terms of the straight line distance. We will need to select these columns and reapply the script. It is important to check that such rows are a minority.

In [6]:
bad_rows_80 = OD_matrix_80.loc[(OD_matrix_80['walk']==True) & (OD_matrix_80['distance'] > 2000)]

In [7]:
bad_rows_80

,safegraph_place_id,census_block_group,top_category,latitude,longitude,fuacode,intptlat,intptlon,walk,origin_node,destination_node,distance
33975,sg:0019073c3a6b4f47b31aae863a1fd8e9,550250113011,"Museums, Historical Sites, and Similar Institu...",43.190772,-89.448029,USA80,43.205964,-89.448195,True,7794027730,2484168381,2235.279
33976,sg:0019073c3a6b4f47b31aae863a1fd8e9,550250113022,"Museums, Historical Sites, and Similar Institu...",43.190772,-89.448029,USA80,43.184520,-89.431324,True,2521835313,2484168381,2152.622
33979,sg:0020ba99d61547aeb8dc4984409df935,550250006003,"Museums, Historical Sites, and Similar Institu...",43.028188,-89.456800,USA80,43.019631,-89.462452,True,5569162367,5368464948,3488.850
33990,sg:00235c5b5ae4403abfa1fd20ccb69d01,550250012004,Restaurants and Other Eating Places,43.074940,-89.390787,USA80,43.064874,-89.404903,True,5489150623,3269859225,2226.806
33991,sg:0025699830dc410eaa82844eeeb8b360,550250119004,"Museums, Historical Sites, and Similar Institu...",42.991451,-89.022372,USA80,42.984409,-89.028839,True,8124599136,2457606095,2016.856
...,...,...,...,...,...,...,...,...,...,...,...,...
39470,sg:98fcd66aa3da4e25acfea23a1bdc6c10,550250109011,Restaurants and Other Eating Places,43.075378,-89.528439,USA80,43.065130,-89.547971,True,5483831985,443884009,2602.998
39472,sg:99129e09dffb46029ff3cc6ff49e68e9,550250016044,Restaurants and Other Eating Places,43.067537,-89.408167,USA80,43.076617,-89.395020,True,2705034726,3811347658,2032.864
39487,sg:991b900dca4948bbae8cf4e7ea77bca4,550250031001,Health and Personal Care Stores,43.085170,-89.277655,USA80,43.081103,-89.296367,True,53476533,4184204415,2074.149
39490,sg:993bb6220d964c2795792c144d998c93,551110001001,Religious Organizations,43.625231,-89.773420,USA80,43.617606,-89.789501,True,5754368622,1347285687,4146.551


How many rows do these represent in terms of all rows? Of all walking rows?

In [8]:
len(bad_rows_80)/len(OD_matrix_80)

0.03587977616286431

In [9]:
len(bad_rows_80)/len(OD_matrix_80.loc[OD_matrix_80['walk']])

0.25679594055817323

Now for the 81:

In [10]:
bad_rows_81 = OD_matrix_81.loc[(OD_matrix_81['walk']==True) & (OD_matrix_81['distance'] > 2000)]
print(len(bad_rows_81)/len(OD_matrix_81))
print(len(bad_rows_81)/len(OD_matrix_81.loc[OD_matrix_81['walk']]))

0.044413290340670125
0.4601975595583963


Although the bad rows represent a significant share of the walking patterns, they represent a very negligible share of the total patterns and thus have low computational cost. There is not many ways to improve on this filtering method, and we can see that in fact the amount of "bad" walking patterns with slightly too long distance is also very significant:

In [11]:
print(len(OD_matrix_80.loc[(OD_matrix_80['walk']==True) & (OD_matrix_80['distance'] > 2000) & (OD_matrix_80['distance'] < 2500)])/len(bad_rows_80))
print(len(OD_matrix_81.loc[(OD_matrix_81['walk']==True) & (OD_matrix_81['distance'] > 2000) & (OD_matrix_81['distance'] < 2500)])/len(bad_rows_81))

0.6224417784050812
0.4476010101010101


i.e. about half of the badly labeled walking patterns have a distance between 2 and 2.5km, which is just slightly above the intended 2km threshold.

## 4. Getting the Final Distance

The following function does the job of getting the final matrix:

In [12]:
def get_final_OD(fua_code, threshold=2000, number_of_cores=1):

    #Get the files:
    od_matrix = pd.read_csv('../data/d04_final-OD-matrices/naive-OD-per-FUA/'+fua_code+'_full-ODmatrix.csv')
    drive_graph = ox.project_graph(ox.load_graphml('../data/d03_intermediate/FUA-networks/drive/'+fua_code+'.graphml'), to_crs='EPSG:5070')
    print('  got all files')

    #Get the geometries of origin and destinations:
    places_pt = gpd.points_from_xy(x=od_matrix.longitude, y=od_matrix.latitude, crs='EPSG:4326').to_crs('EPSG:5070')
    centroids_pt = gpd.points_from_xy(x=od_matrix.intptlon, y=od_matrix.intptlat, crs='EPSG:4326').to_crs('EPSG:5070')

    od_matrix['origin_x'], od_matrix['origin_y'] = centroids_pt.x, centroids_pt.y
    od_matrix['dest_x'], od_matrix['dest_y'] = places_pt.x, places_pt.y

    #Get the rows that need reworking:
    bad_rows = (od_matrix['walk']==True) & (od_matrix['distance'] > threshold)
    print('  got bad rows')

    #Set the Boolean value of whether we walk or drive to False in the bad rows:
    od_matrix.loc[bad_rows, 'walk'] = False

    #We do nearest nodes from OSMnx on the driving graph and the distance for those rows:
    od_matrix.loc[bad_rows, 'origin_node'] = ox.nearest_nodes(drive_graph,
                                                              od_matrix.loc[bad_rows, 'origin_x'], od_matrix.loc[bad_rows, 'origin_y'])
    od_matrix.loc[bad_rows, 'destination_node'] = ox.nearest_nodes(drive_graph,
                                                                   od_matrix.loc[bad_rows, 'dest_x'], od_matrix.loc[bad_rows, 'dest_y'])
    od_matrix.loc[bad_rows, 'distance'] = shortest_path_distance(drive_graph,
                                                                 od_matrix.loc[bad_rows, 'origin_node'].values, od_matrix.loc[bad_rows, 'destination_node'].values,
                                                                 cpus=number_of_cores)

    final_od_matrix = od_matrix.drop(['Unnamed: 0', 'origin_x', 'origin_y', 'dest_x', 'dest_y'], axis=1)
    print('  got new OD matrix')

    final_od_matrix.to_csv('../data/d04_final-OD-matrices/final-OD-per-FUA/' + fua_code+'_final-ODmatrix.csv')
    print('  saved')
    
    return final_od_matrix

Let's do it for two of our FUAs to check:

In [13]:
final_od_matrix_80 = get_final_OD('USA80')
print('\n')
final_od_matrix_81 = get_final_OD('USA81')

  got all files
  got bad rows
  got new OD matrix
  saved


  got all files
  got bad rows
  got new OD matrix
  saved


Do they have any more bad rows?

In [14]:
any_bad_rows_80 = final_od_matrix_80.loc[(final_od_matrix_80['walk']==True) & (final_od_matrix_80['distance'] > 2000)]
any_bad_rows_81 = final_od_matrix_81.loc[(final_od_matrix_81['walk']==True) & (final_od_matrix_81['distance'] > 2000)]

In [15]:
any_bad_rows_80

,safegraph_place_id,census_block_group,top_category,latitude,longitude,fuacode,intptlat,intptlon,walk,origin_node,destination_node,distance


In [16]:
any_bad_rows_81

,safegraph_place_id,census_block_group,top_category,latitude,longitude,fuacode,intptlat,intptlon,walk,origin_node,destination_node,distance
